<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Grid-information" data-toc-modified-id="Grid-information-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Grid information<a class="anchor" id="0_Grid_information" rel="nofollow"></a></a></span></li><li><span><a href="#Definitions" data-toc-modified-id="Definitions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Definitions</a></span></li><li><span><a href="#Import-grid" data-toc-modified-id="Import-grid-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Import grid</a></span></li><li><span><a href="#Plots" data-toc-modified-id="Plots-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Plots</a></span><ul class="toc-item"><li><span><a href="#curtailment" data-toc-modified-id="curtailment-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>curtailment</a></span></li><li><span><a href="#Slacks" data-toc-modified-id="Slacks-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Slacks</a></span></li><li><span><a href="#Energy" data-toc-modified-id="Energy-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Energy</a></span></li><li><span><a href="#Charging" data-toc-modified-id="Charging-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Charging</a></span></li><li><span><a href="#Charging-+-feedin" data-toc-modified-id="Charging-+-feedin-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Charging + feedin</a></span></li></ul></li><li><span><a href="#Plot-dumb-+-opt-charging" data-toc-modified-id="Plot-dumb-+-opt-charging-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Plot dumb + opt charging</a></span></li><li><span><a href="#Plot-energy-level-+-charging" data-toc-modified-id="Plot-energy-level-+-charging-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Plot energy level + charging</a></span></li></ul></div>

# Imports

In [ ]:
######## LOBAFLEX ###########
# from lobaflex.analysis import analysis
# from lobaflex.tools.draw import *
from lobaflex.opt.feeder_extraction import get_flexible_loads
from lobaflex.tools.tools import get_config, get_files_in_subdirs
from lobaflex.opt.dispatch_integration import integrate_dispatch
from edisgo.edisgo import import_edisgo_from_files

######## DIRS ##########
from lobaflex import config_dir, data_dir, results_dir
######## DATA ###########
import re
import numpy as np
import pandas as pd
from pathlib import Path
######## PLOTTING ###########
import matplotlib
# matplotlib.use('TkAgg')
# %matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import ScalarFormatter
import seaborn as sns

# # set seaborn style
# sns.set()
# plotly
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio


from lobaflex.tools.plots import (get_all_attribute_values_for_keyword,
                                  plot_all_attributes_for_keyword,
                                  heatmap_energy_level,
                                 )
# comment to make plots interactive
# pio.renderers.default = "svg"



In [ ]:
pio.renderers.default = "svg"

In [ ]:
cfg_o = get_config(path=config_dir / ".opt.yaml")
# cfg_g = get_config(path=config_dir / ".grids.yaml")

# Grid information<a class="anchor" id="0_Grid_information"></a>

# Definitions

In [ ]:
grid_path = Path(import_dir) / "mvgd"
base_objective =  grid_path.parent.name
results_path = results_dir / run_id / str(grid_id) / "potential" / base_objective
list_of_results = get_files_in_subdirs(path=results_path, pattern="*.csv")

df_observation_periods = pd.read_csv(results_dir / "observation_periods.csv",
                                     header=[0,1], index_col=0).loc[:,str(grid_id)]

# Import grid

In [ ]:
edisgo_obj = import_edisgo_from_files(
    grid_path,
    import_topology=True,
    import_timeseries=True,
    import_heat_pump=True,
    import_electromobility=True,
)

# Plots
## curtailment

In [ ]:
plot_all_attributes_for_keyword(results_path=results_path,
                                keyword="curtailment",
                                timeframe=df_observation_periods[period])

## Slacks

In [ ]:
plot_all_attributes_for_keyword(results_path=results_path,
                                keyword="slack",
                               timeframe=df_observation_periods[period])

## Energy

In [ ]:
plot_all_attributes_for_keyword(results_path=results_path,
                                keyword=f"{grid_id}_energy_level",
                                edisgo_obj=edisgo_obj,
                               timeframe=df_observation_periods[period])

heatmap

- tes upper energy level constraint needs to be removed

## Charging

- diff zu pset hp
- diff zu upper power ev

charging ev: hüllkurve upper lower opacitiy und dann opt ergebnis dazwischen
hp: same

annotation for max min grid power: no tes energy constraints

min max energy level - what happens at peaks? feedin und demand plots?

In [ ]:
plot_all_attributes_for_keyword(results_path=results_path,
                                keyword="charging", edisgo_obj=edisgo_obj,
                               timeframe=df_observation_periods[period])

- high initial charging bei step 1
    - hp
    - tes
- Speicher werden direkt voll gemacht

- konstantes laden der TES wegen verlusten
- warum Einbruch am Ende?

## Charging + feedin

In [ ]:
plot_optimized_dispatch(edisgo_obj, timeframe=df_observation_periods[period])

# Plot dumb + opt charging

# Plot energy level + charging